<a href="https://colab.research.google.com/github/AndrewZinc/Big_Data/blob/main/Stop_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
print('hello1')
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
print('hello2')
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
print('hello3')
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()
print('hello4')

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
hello1
hello2
hello3
hello4


In [8]:
# Create a sample dataframe

sentenceData = spark.createDataFrame([
    (0, ['Spark', 'is', 'great']),
    (1, ['We', 'are', 'learning', 'about', 'Spark']),
    (2, ['Spark', 'is', 'no', 'doubt', 'better', 'than', 'Hadoop'])
    ], ['id', 'raw'])

sentenceData.show(truncate=False)

+---+--------------------------------------------+
|id |raw                                         |
+---+--------------------------------------------+
|0  |[Spark, is, great]                          |
|1  |[We, are, learning, about, Spark]           |
|2  |[Spark, is, no, doubt, better, than, Hadoop]|
+---+--------------------------------------------+



In [9]:
# Import stop words library

from pyspark.ml.feature import StopWordsRemover

# Run the remover
remover = StopWordsRemover(inputCol='raw', outputCol='filtered')

# Transform and show the data
remover.transform(sentenceData).show(truncate=False)

+---+--------------------------------------------+------------------------------+
|id |raw                                         |filtered                      |
+---+--------------------------------------------+------------------------------+
|0  |[Spark, is, great]                          |[Spark, great]                |
|1  |[We, are, learning, about, Spark]           |[learning, Spark]             |
|2  |[Spark, is, no, doubt, better, than, Hadoop]|[Spark, doubt, better, Hadoop]|
+---+--------------------------------------------+------------------------------+



In [10]:
# Create a sample dataframe

df = spark.createDataFrame([
    (0, 'Spark is great'),
    (1, 'We are learning about Spark'),
    (2, 'Spark is no doubt better than Hadoop')
    ], ['id', 'sentence'])

df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning a...|
|  2|Spark is no doubt...|
+---+--------------------+



In [12]:
# Import the tokenizer library
from pyspark.ml.feature import Tokenizer

# tokenize the sentences
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

# Transform and show the DataFrame
tokenized_df = tokenizer.transform(df)
tokenized_df.show(truncate=False)

+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning about Spark         |[we, are, learning, about, spark]           |
|2  |Spark is no doubt better than Hadoop|[spark, is, no, doubt, better, than, hadoop]|
+---+------------------------------------+--------------------------------------------+



In [13]:
# Run the remover
removel = StopWordsRemover(inputCol='words', outputCol='filtere')

# Transform and show the data
removel.transform(tokenized_df).show(truncate=False)

+---+------------------------------------+--------------------------------------------+------------------------------+
|id |sentence                            |words                                       |filtere                       |
+---+------------------------------------+--------------------------------------------+------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |[spark, great]                |
|1  |We are learning about Spark         |[we, are, learning, about, spark]           |[learning, spark]             |
|2  |Spark is no doubt better than Hadoop|[spark, is, no, doubt, better, than, hadoop]|[spark, doubt, better, hadoop]|
+---+------------------------------------+--------------------------------------------+------------------------------+

